In [1]:
from utils.tushare_utils  import pro,ts
df = pro.stock_basic(
                exchange='',
                list_status='L',
                fields='ts_code,name'
            )
name = df[df['ts_code']=='000002.SZ']['name'][1]
print(ts.realtime_quote(ts_code='600000.SH', src='dc'))

没有找到 trade_date 为 20250829 的记录
   NAME    TS_CODE      DATE       TIME   OPEN  PRE_CLOSE  PRICE   HIGH  \
0  浦发银行  600000.SH  20250829   16:11:42  13.81      13.86  13.63  13.99   

     LOW    BID  ...  A1_V   A1_P    A2_V   A2_P  A3_V  A3_P    A4_V   A4_P  \
0  13.63  13.63  ...  30.0  13.68  2387.0  13.69  72.0  13.7  2469.0  13.71   

   A5_V   A5_P  
0  50.0  13.72  

[1 rows x 33 columns]


In [ ]:
from utils.tushare_utils import IndexAnalysis
IndexAnalysis.rt_min('000001.SH',5)

In [18]:
from utils.tushare_utils import IndexAnalysis


import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
def calculate_rsi_tonghuashun(data, period=14):
    """
    计算与同花顺匹配的RSI值（使用Wilder平滑方法）
    
    参数:
    data: DataFrame, 包含价格数据，必须有'close'列
    period: int, RSI计算周期（6或12）
    
    返回:
    float: 最新一条数据的RSI值
    """
    # 确保有足够的数据点
    if len(data) < period + 1:
        return None
    
    # 复制数据以避免修改原始DataFrame
    df = data.copy()
    
    # 计算价格变化
    delta = df['close'].diff()
    
    # 分离上涨和下跌
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # 计算初始平均值（前period个周期的简单平均）
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # 使用Wilder平滑方法计算后续平均值
    for i in range(period, len(df)):
        avg_gain.iloc[i] = (avg_gain.iloc[i-1] * (period - 1) + gain.iloc[i]) / period
        avg_loss.iloc[i] = (avg_loss.iloc[i-1] * (period - 1) + loss.iloc[i]) / period
    
    # 计算相对强度(RS)
    rs = avg_gain / avg_loss
    
    # 计算RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi.iloc[-1]  # 返回最新值

def get_tonghuashun_rsi_values(data):
    """
    计算同花顺风格的RSI6和RSI12值
    
    参数:
    data: DataFrame, 包含价格数据，必须有'close'列
    
    返回:
    dict: 包含RSI6和RSI12值的字典
    """
    # 计算RSI6
    rsi6 = calculate_rsi_tonghuashun(data, period=6)
    
    # 计算RSI12
    rsi12 = calculate_rsi_tonghuashun(data, period=12)
    
    return {
        "RSI6": round(rsi6, 2) if rsi6 is not None else None,
        "RSI12": round(rsi12, 2) if rsi12 is not None else None
    }
print(get_tonghuashun_rsi_values(IndexAnalysis.rt_min('399006.SZ',30)))

{'RSI6': 23.9, 'RSI12': 32.48}
